In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [3]:
import pandas as pd 
import numpy as np
import pickle 
import seaborn as sns 
import matplotlib.pyplot as plt  

from sklearn.feature_extraction import DictVectorizer  
from sklearn.linear_model import LinearRegression  
from sklearn.linear_model import Lasso  
from sklearn.linear_model import Ridge  

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import root_mean_squared_error 

/Users/optimistix/anaconda3/envs/ml-zoomcamp/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

## Q1. Notebook

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

In [8]:
np.std(y_pred)

6.247488852238703

### Thus, the standard deviation of the predicted duration for this dataset is <b>6.24</b>.

## Q2. Preparing the output

In [9]:
taxi_type = 'yellow'
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame()
df_result["ride_id"] = df["ride_id"]
df_result["predicted_duration"] = y_pred

In [11]:
output_file = f"{taxi_type}-{year:04d}-{month:02d}.parquet"

In [12]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [13]:
!du -k yellow-2023-03.parquet

81924	yellow-2023-03.parquet


### Thus, the output file is <b>66 M</b>. 